# Using a Gradio app with a Dialogflow CX Virtual Agent

This notebook will create a new Gradio app that hooks into an existing chatbot running on Dialogflow CX in Google Cloud.

### Imports!

In [ ]:
import argparse
import uuid
import random
import time

import gradio as gr

from google.cloud import dialogflowcx_v3beta1 as dialogflow
from google.cloud.dialogflowcx_v3beta1.services.agents import AgentsClient
from google.cloud.dialogflowcx_v3beta1.services.sessions import SessionsClient
from google.cloud.dialogflowcx_v3beta1.types import session

### Set the following variables to your Google Cloud project ID, Dialogflow CX agent region, Dialogflow CX agent ID, and language

In [ ]:
PROJECT_ID = "your-project-id"    # https://cloud.google.com/resource-manager/docs/creating-managing-projects
LOCATION_ID = "your-location-id"  # https://cloud.google.com/dialogflow/cx/docs/concept/region
AGENT_ID = "your-agent-id"        # https://stackoverflow.com/questions/65389964/where-can-i-find-the-dialogflow-cx-agent-id
LANGUAGE_CODE = "en-us"           # https://cloud.google.com/dialogflow/cx/docs/reference/language

### Create a session with the Dialogflow CX virtual agent

In [ ]:
def create_session(agent, session_id):
    session_path = f"{agent}/sessions/{session_id}"
    print(f"Session path: {session_path}\n")
    client_options = None
    agent_components = AgentsClient.parse_agent_path(agent)
    location_id = agent_components["location"]
    if location_id != "global":
        api_endpoint = f"{location_id}-dialogflow.googleapis.com:443"
        print(f"API Endpoint: {api_endpoint}\n")
        client_options = {"api_endpoint": api_endpoint}
    session_client = SessionsClient(client_options=client_options)
    return session_client

### Given text input from the end-user, detect the intent and return a response from the Dialogflow CX agent

In [ ]:
def detect_intent_texts(agent, session_id, session_client, texts, language_code):
    """Returns the result of detect intent with texts as inputs.
    Using the same `session_id` between requests allows continuation
    of the conversation."""
    session_path = f"{agent}/sessions/{session_id}"
    for text in texts:
        text_input = session.TextInput(text=text)
        query_input = session.QueryInput(text=text_input, language_code=language_code)
        request = session.DetectIntentRequest(
            session=session_path, query_input=query_input
        )
        response = session_client.detect_intent(request=request)

        response_messages = [
            " ".join(msg.text.text) for msg in response.query_result.response_messages
        ]
        
        # print("=" * 20)
        # print(f"Query text: {response.query_result.text}")
        # print(f"Response text: {' '.join(response_messages)}\n")

        return(f"{' '.join(response_messages)}\n")

### Create a reusable session with the Dialogflow CX virtual agent

In [ ]:
agent = f"projects/{PROJECT_ID}/locations/{LOCATION_ID}/agents/{AGENT_ID}"
session_id = uuid.uuid4()
current_session = create_session(agent, session_id)

### Create a chatbot app with Gradio and integrate it with Dialogflow CX

In [ ]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Gradio + Dialogflow CX Chatbot")
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def respond(message, chat_history):
        texts = [message]
        bot_message = detect_intent_texts(agent, session_id, current_session, texts, LANGUAGE_CODE)
        chat_history.append((message, bot_message))
        time.sleep(1)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

### Launch the Gradio app and start chatting with the Dialogflow CX virtual agent!

In [ ]:
demo.launch()